# Riza Code Interpreter

> Riza Code Interpreter 是一个基于 WASM 的隔离沙箱环境，用于运行由 AI 代理生成的 Python 或 JavaScript 代码。

在本 Notebook 中，我们将创建一个代理示例，该代理将使用 Python 来解决 LLM 无法独立完成的问题：
计算单词 "strawberry" 中字母 "r" 的数量。

开始之前，请从 [Riza dashboard](https://dashboard.riza.io) 获取你的 API 密钥。如需更多指南和完整的 API 参考，请访问 [Riza Code Interpreter API documentation](https://docs.riza.io)。

请确保已安装必要的依赖项。

In [ ]:
%pip install --upgrade --quiet langchain-community rizaio

将 API 密钥设置为环境变量。

In [ ]:
%env ANTHROPIC_API_KEY=<your_anthropic_api_key_here>
%env RIZA_API_KEY=<your_riza_api_key_here>

In [13]:
from langchain_community.tools.riza.command import ExecPython

In [14]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate

初始化 `ExecPython` 工具。

In [15]:
tools = [ExecPython()]

使用 Anthropic 的 Claude Haiku 模型初始化一个 agent。

In [16]:
llm = ChatAnthropic(model="claude-3-haiku-20240307", temperature=0)

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Make sure to use a tool if you need to solve a problem.",
        ),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

agent = create_tool_calling_agent(llm, tools, prompt_template)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [34]:
# Ask a tough question
result = agent_executor.invoke({"input": "how many rs are in strawberry?"})
print(result["output"][0]["text"])



> Entering new AgentExecutor chain...

Invoking: `riza_exec_python` with `{'code': 'word = "strawberry"\nprint(word.count("r"))'}`
responded: [{'id': 'toolu_01JwPLAAqqCNCjVuEnK8Fgut', 'input': {}, 'name': 'riza_exec_python', 'type': 'tool_use', 'index': 0, 'partial_json': '{"code": "word = \\"strawberry\\"\\nprint(word.count(\\"r\\"))"}'}]

3
[{'text': '\n\nThe word "strawberry" contains 3 "r" characters.', 'type': 'text', 'index': 0}]

> Finished chain.


The word "strawberry" contains 3 "r" characters.
